<a href="https://colab.research.google.com/github/stefanoduri/Opendata_sanita/blob/master/Scraping_Battiti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SETUP DI SELENIUM
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#import sys
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install python3-discogs-client


In [ ]:
# SETUP DI SELENIUM

#set options to be headless
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import html
from datetime import datetime
import discogs_client
import time



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Lettura dei dati dei podcast

In [ ]:
brani_file = "/content/drive/MyDrive/Colab Notebooks/output/Battiti/brani.csv"
puntate_file = "/content/drive/MyDrive/Colab Notebooks/output/Battiti/puntate.csv"
artisti_file = "/content/drive/MyDrive/Colab Notebooks/output/Battiti/artisti.csv"

In [ ]:
# lettura brani per individuare max data
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

brani = pd.read_csv(brani_file, sep="\t")
puntate = pd.read_csv(puntate_file, sep="\t",parse_dates=["date"], date_parser=dateparse)
max_date = puntate.date.max().strftime("%Y%m%d")


print(max_date)


In [ ]:
podcasts = pd.DataFrame(columns = ['date', 'puntata', 'url'])
baseUrl ="https://www.raiplayradio.it/programmi/battiti/archivio/puntate/2021--a25f2341-c42a-4cc2-98c1-329375b9908d/"
page = 1

driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(2)
driver.get(f"{baseUrl}{page}")
driver.maximize_window()

# accetta cookies
try:
    WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.XPATH, "//button[@data-context='YES']"))
    )
    acceptCookies = driver.find_element_by_xpath("//button[@data-context='YES']")
    print(acceptCookies.text)
    actions = ActionChains(driver)
    actions.move_to_element(acceptCookies)
    actions.click(acceptCookies)
    actions.perform()
except Exception as e:
    print("Err "+str(e))


In [ ]:
print(driver.current_url)

In [ ]:
new = True    
while new == True:
    try:          
        podcastsEl = WebDriverWait(driver,10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='columns large-expand medium-expand small-12']"))
        )
        #print("---",podcastsEl)
        
        for e in podcastsEl:
            p = {}
            p['date'] = e.find_element_by_tag_name("span").text
            serialDate = p['date'][-4:]+p['date'][3:5]+p['date'][0:2]
            print(serialDate)
            if serialDate <= max_date:
              new = False
              break
            else:
              p['url'] = e.find_element_by_tag_name("a").get_attribute("href")
              p['puntata'] = e.find_element_by_tag_name("a").text
              podcasts = podcasts.append(p, ignore_index=True)
        if new:
          page+=1
          time.sleep(1)
          driver.get(f"{baseUrl}{page}")
    except Exception as e:
        print("Err 2 - "+str(e))
        break
print(podcasts.describe())


### Lettura dei singoli podcast

In [ ]:
tracks = pd.DataFrame()
i=0
for index, p in podcasts.iterrows():
    driver.get(p['url'])
    print(p['date'])
    try:
      htmlList = driver.find_element_by_class_name("aodHtmlDescription").\
          get_property("innerHTML").split('<br>')
      htmlList = [re.sub("\n","",html.unescape(x)).strip() for x in htmlList]
    except Exception as e:
      print("Errore lettura htmlList "+e)
      continue
    #print(htmlList)
    track = p
    for trackData in htmlList:
        trackData = re.sub('[“”"]','',trackData)
        try:
            trackDataParts = re.split(", *da *?", trackData)
            match = re.match('^([^,]+?),\s*(.+?)$', trackDataParts[0].strip())
            track['artist'] = match[1]
            track['track'] = match[2]
            
            album = re.split("[-–]", trackDataParts[1])
            track['album'] = album[0].strip()
            track['label'] = album[1].strip()
            tracks = tracks.append(track, ignore_index=True)
            #print("tipo 1", trackData)
            
        except:
            try:
                trackDataParts = re.split("',\s*'", trackData)
                trackDataParts[0] = re.sub("'", "", trackDataParts[0].strip())
                trackDataParts[1] = re.sub("'", "", trackDataParts[1].strip())
                
                match = re.match('^([^,]+?),\s*(.+?)$', trackDataParts[0])
                track['artist'] = match[1].strip()
                track['track'] = match[2].strip()

                album = re.split("[-–]", trackDataParts[1])
                track['album'] = album[0].strip()
                track['label'] = album[1].strip()
                tracks = tracks.append(track, ignore_index=True)
                print(track)
            except:
                print(trackData)
                continue

In [ ]:
driver.quit()
print(tracks.describe())

### Salvataggio dati

In [ ]:
brani = pd.concat([brani, tracks[["artist", "album","track","label","puntata"]]],ignore_index=True)
brani.to_csv(brani_file, sep="\t", index=False, encoding="utf-8")


In [ ]:
# rilettura di puntate senza date parsing
puntate = pd.read_csv(puntate_file, sep="\t")
nuove_puntate = tracks[["date","puntata","url"]].drop_duplicates(["date","puntata","url"], ignore_index=True)
puntate = pd.concat([puntate, nuove_puntate],ignore_index=True)
puntate.to_csv(puntate_file, sep="\t", index=False, encoding="utf-8")
# N.B. salta le puntate nelle quali non è stato riconosciuto nessun brano

### Url degli artisti

In [ ]:
artisti_salvati = pd.read_csv(artisti_file, sep="\t")
artisti_in_brani = brani[["artist"]].drop_duplicates(["artist"])
artisti_in_brani = artisti_in_brani[~artisti_in_brani["artist"].isin(artisti_salvati["artist"])]
artisti_in_brani["profile"] = ""
artisti_in_brani.head()

In [ ]:
token = "qdZnxTKUKteXowuZeBjmiIbgaupgOrThxwEzuunM"
d = discogs_client.Client('ExampleApplication/0.1', user_token=token)

In [ ]:
for i, r in artisti_in_brani.iterrows():
  
  results = d.search(r.artist, type='artist')
  time.sleep(1)

  if results:
    r["profile"] = "https://www.discogs.com/en"+results.page(0)[0].url+"?lang_alt=en"
    print(f"{r.artist} {r.profile}")
    

In [ ]:
artisti_in_brani = pd.concat([artisti_salvati, artisti_in_brani], ignore_index=True)

In [ ]:
mancanti = artisti_in_brani.query("profile==''")
print(mancanti.describe())
da_cercare = pd.DataFrame(columns=mancanti.columns)
delim = '\s*\/\s*| WITH | with | \& |\s*\+\s*'

for i,r in mancanti.iterrows():
  split_artist = list(map(lambda x:x.strip(),re.split(delim, r["artist"])))
  if len(split_artist) > 1:
    for a in split_artist:
        if artisti_in_brani.query(f"artist=='{a}'").empty:
          results = d.search(a, type='artist')
          time.sleep(1)

          if results:
            new = {}
            new["artist"] = a
            new["profile"] = "https://www.discogs.com/en"+results.page(0)[0].url+"?lang_alt=en"
            artisti_in_brani = artisti_in_brani.append(new,ignore_index=True)
            print(f"{new['artist']} {new['profile']}")

In [ ]:
print(len(artisti_in_brani))
artists_with_profile = artisti_in_brani.query("profile>''")
print(len(artists_with_profile))
artists_with_profile.to_csv(artisti_file, sep="\t", index=False, encoding="utf-8")
